In [1]:
import json
import pandas as pd

In [3]:
#This function takes the file path to the json file as a parameter and returns a dataframe with 3 columns;
# ARTICLE_LINK,HEADLINE and ISARCASTIC

def convert_2_df(filr_path):
    filr_handlr = open(filr_path,'r')
    content = filr_handlr.read().split('\n')
    container =[]
    for row in content:
        if len(row) == 0:
            continue
        json_to_dict = json.loads(row)
        item = [i for i in json_to_dict.values()]
        container.append(item)
    df = pd.DataFrame(container,columns=['ARTICLE_LINK','HEADLINE','IS_SARCASTIC'])
    return df

In [4]:
df = convert_2_df('Sarcasm_Headlines_Dataset.json')
df.head()

,ARTICLE_LINK,HEADLINE,IS_SARCASTIC
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [26]:
#This function takes in a sentence, tokenizes it into a list of words,
#finds the lemma for each word and returns them as a list.

def token_lemmatizer(text):
    wn = WordNetLemmatizer()
    tokens =[]
    for pos in pos_tag(word_tokenize(text)):
        w = pos[0]
        if pos[1][0] == 'N':
            lemma = wn.lemmatize(w,pos='n')
        elif pos[1][0] == 'V':
            lemma = wn.lemmatize(w,pos='v')
        elif pos[1][0] == 'R':
            lemma = wn.lemmatize(w,pos='r')
        elif pos[1][0] == 'J':
            lemma =  wn.lemmatize(w,pos='a')
        else:
            lemma = wn.lemmatize(w)
        tokens.append(lemma)
    return tokens

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [8]:
english_stop_words = stopwords.words("english")

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['HEADLINE'], df['IS_SARCASTIC'], 
                                                    test_size=0.3, random_state=101)

## Using the Training data, i find the best parameters for the Count Vectorizer.

In [38]:
count_vectorizer = CountVectorizer()
count_vectorizer_pipe = ('count_vectorizer',count_vectorizer)

In [39]:
param_grid = {
    'count_vectorizer__ngram_range': [(1,1),(1,2), (1,3),(1,4)],
    'count_vectorizer__lowercase':[True,False],
    'count_vectorizer__stop_words':[None,english_stop_words],
    'count_vectorizer__tokenizer':[word_tokenize,token_lemmatizer]
    }

In [40]:
clf_log = LogisticRegression(solver='lbfgs',max_iter=10000)
clf_log_pipe = ('clf_log',clf_log)
pipe_components = [count_vectorizer_pipe,clf_log_pipe]

In [41]:
pipe = Pipeline(pipe_components)
gs = GridSearchCV(pipe, param_grid, cv=5)

In [42]:
gs.fit(X_train,y_train)

/Users/tom/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Users/tom/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Users/tom/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'ne

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
 ...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'count_vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)], 'count_vectorizer__lowercase': [True, False], 'count_vectorizer__stop_words': [None, ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',...tokenizer': [<function word_tokenize at 0x1a236e8598>, <function token_lemmatizer at 0x1a243e6d90>]},
       pre_dispatch='2*n_jobs', refit

In [43]:
gs.best_params_

{'count_vectorizer__lowercase': True,
 'count_vectorizer__ngram_range': (1, 2),
 'count_vectorizer__stop_words': None,
 'count_vectorizer__tokenizer': <function nltk.tokenize.word_tokenize(text, language='english', preserve_line=False)>}

In [44]:
gs.best_score_

0.8638211382113821

### For Count Vectorizer, the best parameters are:
1. lowercase: True
2. NGram Range: (1,2)
3. Stop Words: Nobe
4. Tokenizer: word_tokenize from nltk

In [45]:
mcv = CountVectorizer(lowercase=True,
                                   ngram_range=(1,2),
                                   stop_words=None,
                                   tokenizer=word_tokenize)
mcv_pipe = ('count_vectorizer',mcv)

## Experimenting with L1 and L2 regularization to find the best solver for eact penalty.

In [52]:
param_grid2 = {
    'clf_log2__solver':['newton-cg', 'sag','lbfgs']
    }

In [53]:
#l2: ‘newton-cg’, ‘sag’ and ‘lbfgs’ 
clf_log2 = LogisticRegression(penalty='l2',max_iter=10000)
clf_log_pipe2 = ('clf_log2',clf_log2)
pipe_components2 = [mcv_pipe,clf_log_pipe2]

In [54]:
pipe2 = Pipeline(pipe_components2)
gs2 = GridSearchCV(pipe2, param_grid2, cv=5)

In [55]:
gs2.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
 ...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf_log2__solver': ['newton-cg', 'sag', 'lbfgs']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [56]:
print(gs2.best_params_)

{'clf_log2__solver': 'newton-cg'}


In [57]:
print(gs2.best_score_)

0.8638211382113821


### Using L2 regularization, the best solver is newton-cg


In [58]:
#l1: 'liblinear','saga'
param_grid3 = {
    'clf_log3__solver':['liblinear', 'saga']
    }

In [61]:
clf_log3 = LogisticRegression(penalty='l1',max_iter=10000)
clf_log_pipe3 = ('clf_log3',clf_log3)
pipe_components3 = [mcv_pipe,clf_log_pipe3]

In [62]:
pipe3 = Pipeline(pipe_components3)
gs3 = GridSearchCV(pipe3, param_grid3, cv=5)

In [63]:
gs3.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
 ...penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf_log3__solver': ['liblinear', 'saga']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [64]:
print(gs3.best_params_)

{'clf_log3__solver': 'saga'}


In [65]:
print(gs3.best_score_)

0.8495400085579803


### Using L1 regularization, the best solver is saga

In [66]:
clf_l1 = LogisticRegression(penalty='l1',max_iter=10000,solver='saga')
clf_l2 = LogisticRegression(penalty='l2',max_iter=10000,solver='newton-cg')
clf_cv = CountVectorizer(lowercase=True,
                      ngram_range=(1,2),
                      stop_words=None,
                      tokenizer=word_tokenize)

In [67]:
clf_cv.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x1a236e8598>,
        vocabulary=None)

In [68]:
X_train_cv = clf_cv.transform(X_train)
X_test_cv = clf_cv.transform(X_test)

In [70]:
clf_l1.fit(X_train_cv,y_train)
clf_l2.fit(X_train_cv,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [71]:
y_l1 = clf_l1.predict(X_test_cv)
y_l2 = clf_l2.predict(X_test_cv)

In [72]:
from sklearn.metrics import classification_report

In [73]:
print(classification_report(y_test,y_l1))

              precision    recall  f1-score   support

           0       0.88      0.86      0.87      4485
           1       0.82      0.85      0.84      3528

   micro avg       0.85      0.85      0.85      8013
   macro avg       0.85      0.85      0.85      8013
weighted avg       0.86      0.85      0.85      8013



In [74]:
print(classification_report(y_test,y_l2))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4485
           1       0.84      0.86      0.85      3528

   micro avg       0.87      0.87      0.87      8013
   macro avg       0.86      0.86      0.86      8013
weighted avg       0.87      0.87      0.87      8013



## Optimal paramters for Count Vectorizer and Logistic Regression are:
### Count Vectorizer:
    1. lowercase: True
    2. NGram Range: (1,2)
    3. Stop Words: Nobe
    4. Tokenizer: word_tokenize from nltk
    
### Logistic Regression:
    1. Penalty: L2 (Ridge Regression)
    2. Solver: newton-cg
    